In [1]:
# default_exp core

# pico_pi_controller

> A service interface for a Raspberry Pi 2/3 to be monitored & controlled from a CircuitPython capable microcontroller. Use on a RPi 4 requires a solution to enable i2c perpheral support on RPi4.

This module uses `pigpio` to enable a peripheral (slave) device; `pigpiod` must be running.

In [2]:
#hide
#from nbdev.showdoc import *

In [3]:
#export
from platform import node
from os import getloadavg
from sys import byteorder
import time
import pigpio
import atexit

class PPC_Device_Daemon():
    def __init__(self):
        I2C_ADDR=0x13
        IDENTITY = bytearray('ppdd','ascii')
        IDF = bytearray('I','ascii')
        HOS = bytearray('H','ascii')
        LOD = bytearray('L','ascii')
        hostname = bytearray(node(),'utf-8')
        load = bytearray(str(getloadavg()[0]),'utf-8')

        @atexit.register
        def goodbye():
            try:
                e.cancel()
                pi.bsc_i2c(0) # Disable BSC peripheral
                pi.stop()
                print ("bsc_i2c cleanup completed.")
            except:
                print ("bsc_i2c error on cleanup.")
                print('Exiting via atexit')

        pi = pigpio.pi()
        if not pi.connected:
            print("pigpiod not running. Exiting.")
            exit()
        # setup event handler to listen for heartbeat from MCU
        e = pi.event_callback(pigpio.EVENT_BSC, self.i2c_event_handler)
        while True:
            time.sleep(600)
    
    def i2c_event_handler(id, tick):
        global pi

        s, b, d = pi.bsc_i2c(I2C_ADDR)
        if b:
            load = bytearray("{:04.2f}".format(getloadavg()[0]),'utf-8')
            if d[0] == int.from_bytes(IDF, byteorder=byteorder):
                print ("IDF recieved, sending identity:",IDENTITY.decode(),"on tick",tick)
                s, b, d = pi.bsc_i2c(I2C_ADDR,IDENTITY)
            elif d[0] == int.from_bytes(HOS, byteorder=byteorder):
                print ("HOS recieved, sending hostname:",hostname.decode())
                s, b, d = pi.bsc_i2c(I2C_ADDR, bytes([len(hostname)])+hostname)
            elif d[0] == int.from_bytes(LOD, byteorder=byteorder):
                print ("LOD recieved, sending load:",load.decode())
                s, b, d = pi.bsc_i2c(I2C_ADDR,load)
